In [1]:
pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
  Attempting uninstall: typer
    Found existing installation: typer 0.16.0
    Uninstalling typer-0.16.0:
      Successfully uninstalled typer-0.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 2.8.0 re

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("API")

# Testing The Model


In [4]:
from together import Together
#Replace your API from Togather.AI here 

client = Together(api_key=secret_value_0)

response = client.chat.completions.create(
    #model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    model="deepseek-ai/DeepSeek-R1-0528",
    messages=[
      {
        "role": "user",
        "content": "ما هي عاصمة فرسنا?"
      }
    ]
)
print(response.choices[0].message.content)


<think>
حسناً، المستخدم يسأل عن عاصمة "فرسنا". هذا الاسم غير مألوف بالنسبة لي. ربما هناك خطأ في الكتابة أو التهجئة.

لنراجع الاحتمالات: قد يكون المستخدم يقصد "فرنسا" لأن هذا هو الاسم الشائع للدولة الأوروبية. عاصمة فرنسا هي باريس، وهي معلومة معروفة على نطاق واسع.

لكن من المهم التحقق من عدم وجود دولة أخرى باسم مشابه. بعد البحث السريع في قاعدة المعرفة، لا أجد أي دولة مسجلة باسم "فرسنا". ربما المستخدم غير متأكد من التهجئة الصحيحة.

سأفترض أن المستخدم يعني فرنسا وأقدم الإجابة المناسبة. مع ذلك، سأذكر صراحة أنني أفترض ذلك بسبب تشابه الأسماء، وأشير إلى أن "فرسنا" قد تكون خطأ إملائياً. هذا يعطي المستخدم فرصة لتصحيح السؤال إذا كان يقصد شيئاً آخر.

سأجيب بطريقة واضحة ومباشرة، مع تقديم معلومة إضافية عن باريس لتعزيز الفائدة. إذا كان هناك سوء فهم، سيكون المستخدم قادراً على التوضيح في متابعة السؤال.
</think>
يبدو أن هناك خطأ في كتابة اسم الدولة. لا توجد دولة معروفة باسم "فرسنا".

إذا كنت تقصد **فرنسا** (France)، فالعاصمة هي **باريس** (Paris).

باريس هي مدينة شهيرة عالمياً، وتشتهر بمعالمها مثل برج إي

# Data Loading and Cleaning

In [5]:
# ---- [2] تحميل بياناتك وتحويلها إلى SQL ---- #
import pandas as pd
from sqlalchemy import create_engine

x="/kaggle/input/data-syb/SYB_3_3.xlsx"
# افترض أن ملفك اسمه 'birth_data.xlsx'
df = pd.read_excel(x, sheet_name="Relational DB", skiprows=2)
columns_to_drop = df.columns[[0, 2, 4, 6, 8,11]]  # 1,3,5,7,9 حسب ترقيم البشر

# حذف الأعمدة المحددة
df = df.drop(columns=columns_to_drop)

# عرض الأعمدة المتبقية للتأكد
print("الأعمدة المتبقية:")
print(df.columns.tolist())

الأعمدة المتبقية:
['Measure Name ', 'Unit', 'Governorate', 'Sex', 'قيمة المؤشر', 'سنة فترة القياس']


In [6]:
df = df.rename(columns={
    'Measure Name ': 'measure_name',
    'رمز المؤشر': 'indicator_code',
    'Unit': 'unit',
    'Governorate': 'governorate',
    'Sex': 'sex',
    'قيمة المؤشر': 'indicator_value',
    'سنة فترة القياس': 'year',
})


# إنشاء قاعدة بيانات SQLite في الذاكرة
engine = create_engine("sqlite:///:memory:")
df.to_sql("birth_table", engine, index=False)

234

In [7]:
df.head()

,measure_name,unit,governorate,sex,indicator_value,year
0,Registered Live Births,Number,Amman,Male,38445,2023
1,Registered Live Births,Number,Balqa,Male,3842,2023
2,Registered Live Births,Number,Zarqa,Male,9174,2023
3,Registered Live Births,Number,Madaba,Male,2095,2023
4,Registered Live Births,Number,Irbid,Male,17028,2023


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   measure_name     234 non-null    object
 1   unit             234 non-null    object
 2   governorate      234 non-null    object
 3   sex              234 non-null    object
 4   indicator_value  234 non-null    int64 
 5   year             234 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 11.1+ KB


In [9]:
df["governorate"].unique()

array(['Amman', 'Balqa', 'Zarqa', 'Madaba', 'Irbid', 'Mafraq', 'Jarash',
       'Ajloun', 'Karak', 'Tafiela', "Ma'an", 'Aqaba', 'Expatriates'],
      dtype=object)

In [10]:
# تنظيف كل الأعمدة النصية من الفراغات
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()

print("✅ تم تنظيف البيانات (Trim) لكل الأعمدة النصية.")

# إنشاء قاعدة بيانات SQLite
engine = create_engine("sqlite:///:memory:")
df.to_sql("birth_table", engine, index=False)

print("✅ تم تحميل البيانات إلى SQLite")

✅ تم تنظيف البيانات (Trim) لكل الأعمدة النصية.
✅ تم تحميل البيانات إلى SQLite


In [11]:
ARABIC_SCHEMA = """
لديك DataFrame اسمه df يحتوي على بيانات المواليد. الأعمدة والقيم بالإنجليزية فقط.

🔹 الأعمدة:

- measure_name : (مثال: "Registered Live Births")
- unit : (مثال: "Number")
- governorate : (مثال: "Amman", "Irbid", "Zarqa", "Karak", "Ma'an", "Aqaba")
- sex : (مثال: "Male", "Female")
- indicator_value : (القيمة الرقمية المطلوبة)
- year : (مثال: 2021)

---

⚠️ **تعليمات للمساعد الذكي:**

1. **أجب بالكود فقط دون شرح**
2. **استخدم أسماء الأعمدة والقيم بالإنجليزية فقط**
3. استخدم `df` كاسم DataFrame
4. استخدم `.sum()` عند الحاجة لحساب الإجمالي
5. النتيجة تحفظ في متغير اسمه `result`
---

📊 **مثال:**

❓ السؤال: كم عدد المواليد الذكور في إربد عام 2020؟

✅ الكود:

result = df[
    (df['governorate'] == 'Irbid') & 
    (df['sex'] == 'Male') & 
    (df['year'] == 2020) & 
    (df['measure_name'] == 'Registered Live Births')
]['indicator_value'].sum()

---

❓ السؤال: قارن بين مواليد الذكور والإناث في عمان بين 2020 و 2022؟

✅ الكود:

result = df[
    (df['governorate'] == 'Amman') & 
    (df['year'].between(2020, 2022)) & 
    (df['measure_name'] == 'Registered Live Births')
].groupby(['year', 'sex'])['indicator_value'].sum().unstack()

---

أجب على السؤال التالي بالكود فقط مستخدماً القيم بالإنجليزية:
"""


In [12]:
import pandas as pd
import re
from together import Together

def generate_pandas_code(question: str) -> str:
    prompt = f"""
{ARABIC_SCHEMA}



السؤال الحالي: {question}

⚠️ تعليمات:
- أجب بكود Pandas فقط.
- لا تكتب أي شرح أو تعليق.
- الكود بين ```python و ```
"""

    messages = [
        {"role": "system", "content": "أنت مساعد يحول الأسئلة إلى كود Pandas فقط مع مراعاة السياق."},
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        #model="deepseek-ai/DeepSeek-R1-0528",
        messages=messages,
        temperature=0
    )

    raw_output = response.choices[0].message.content.strip()
    return extract_code_from_response(raw_output)

# ---- [3] استخراج الكود ---- #
def extract_code_from_response(llm_output):
    return re.sub(r"```python|```", "", llm_output).strip()

# ---- [4] تنفيذ الكود ---- #
def run_pandas_code(code: str, df):
    local_vars = {"df": df.copy()}
    try:
        exec(code, {}, local_vars)
        return local_vars.get('result', "⚠️ لم يتم العثور على متغير result")
    except Exception as e:
        return f"❌ خطأ في تنفيذ الكود: {e}"

# ---- [5] توليد شرح النتائج ---- #
def explain_results(question: str, code: str, results) -> str:
    prompt = f"""
قم بشرح النتائج التالية للمستخدم العادي بالعربية فقط:

السؤال: {question}

الكود المستخدم:
{code}

النتائج: {results}

اكتب شرحاً من 2 إلى 3 جمل بلغة بسيطة.
"""
    messages = [
        {"role": "system", "content": "أنت خبير بيانات تشرح النتائج للمستخدمين بالعربية المبسطة فقط."},
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        #model="deepseek-ai/DeepSeek-R1-0528",
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages,
        temperature=0.3
    )

    return response.choices[0].message.content.strip()

# ---- [6] واجهة التفاعل ---- #
def terminal_chat(df):
    print("="*50)
    print("مساعد تحليلات Pandas مع سياق التفاعل".center(50))
    print("="*50)

    while True:
        question = input("\nسؤالك: ").strip()

        if question.lower() in ['خرج', 'exit', 'quit']:
            print("👋 إلى اللقاء!")
            break

        print("\n🔍 توليد الكود...")
        code = generate_pandas_code(question)
        print(f"\n📝 الكود:\n{code}")

        print("\n⚙️ التنفيذ...")
        result = run_pandas_code(code, df)
        print("\n📊 النتيجة:")
        print(result)

        print("\n💡 التفسير:")
        explanation = explain_results(question, code, result)
        print(explanation)

if __name__ == "__main__":
   
    terminal_chat(df)

       مساعد تحليلات Pandas مع سياق التفاعل       



سؤالك:  كم عدد مواليد الذكور في عمان 2020؟



🔍 توليد الكود...

📝 الكود:
result = df[
    (df['governorate'] == 'Amman') & 
    (df['sex'] == 'Male') & 
    (df['year'] == 2020) & 
    (df['measure_name'] == 'Registered Live Births')
]['indicator_value'].sum()

⚙️ التنفيذ...

📊 النتيجة:
37580

💡 التفسير:
هذه النتائج تظهر عدد مواليد الذكور في محافظة عمان في سنة 2020، والذي بلغ 37580 مولوداً. هذا الرقم يمثل إجمالي عدد الأطفال الذكور الذين ولدوا في عمان في تلك السنة.



سؤالك:  exit


👋 إلى اللقاء!


In [13]:
#TESTING THE RESULTS
result = df[
    (df['governorate'] == 'Amman') & 
    (df['sex'] == 'Male') & 
    (df['year'] == 2020) & 
    (df['measure_name'] == 'Registered Live Births')
]['indicator_value'].sum()
result

37580

# 2 Tables


In [14]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import gradio as gr
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os
from together import Together
metadata_index = {
    "SYB_3_3.xlsx": {
        "summary": "عدد المواليد الأحياء المسجلين حسب الجنس والمحافظة",
        "path": "/kaggle/input/data-syb/SYB_3_3.xlsx",
    },
    "SYB_3_11.xlsx": {
        "summary": "عقود الزواج المسجلة حسب المحافظة وخصائص مختارة للزوجة",
        "path": "/kaggle/input/data-syb/SYB_3_11.xlsx",
    }
}

def choose_best_table(question: str):
    prompt = "السؤال:\n" + question + "\n\nاختر أنسب جدول من القائمة التالية بناءً على محتوى السؤال:\n"
    for i, (file, meta) in enumerate(metadata_index.items()):
        prompt += f"{i+1}. {meta['summary']} ({file})\n"
    prompt += "\nاكتب فقط اسم الملف الأنسب (مثل: SYB_3_3.xlsx):"

    messages = [
        {"role": "system", "content": "أنت مساعد ذكي تختار الجدول الأنسب للإجابة على سؤال."},
        {"role": "user", "content": prompt}
    ]
    client = Together(api_key=secret_value_0)
    response = client.chat.completions.create(
        #model="deepseek-ai/DeepSeek-R1-0528",
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages,
        temperature=0.3
    )
    answer = response.choices[0].message.content.strip()
    filename = answer.split()[-1] if answer.endswith('.xlsx') else answer
    return filename if filename in metadata_index else list(metadata_index.keys())[0]

def extract_code_from_response(text):
    match = re.search(r"```python(.*?)```", text, re.DOTALL)
    return match.group(1).strip() if match else text.strip()

def generate_pandas_code(question: str, df_name: str = "df", table_info: str = "") -> str:
    prompt = f"""
افترض أن الجدول الحالي اسمه {df_name}
قيمة المؤشر هو عدد الممثل للسطر
مهم جداً: العمود \"قيمة المؤشر\" (قيمة المؤشر) هو القيمة الأساسية والأهم التي يجب التركيز عليها عند التحليل.
معلومات عن الأعمدة والقيم الفريدة (باستثناء والمؤشرات):

{table_info}


السؤال الحالي: {question}

⚠️ تعليمات:
- أجب بكود Pandas .
-لا تظهر التفكير فقط نفذ باحترافية 
-fig1 = plt.figure(figsize=(10, 6)) # Simple assignment
-تاكد من ان تكتب جميع الكودات بشكل صحيح وبلا اخطاء
- اذا كان السؤال بالعربي استخدم الاعمده العربيه والقيم العربية والعكس صحيح ما عدا قيمه المؤشر و السنه مثال: (df['المحافظة'] == 'الزرقاء')
- استخدم df[condition] لتصفية البيانات أولاً.
- استخدم `.sum()` عند الحاجة لحساب الإجمالي.
- يجب وضع الكود بين ```python و ```
- يجب أن تسند النتائج النهائية إلى متغيرات منطقية مثل: result_year، result_gender، result_gov
- وإذا كان الرسم منطقياً، استخدم متغيرات fig1, fig2, ... لحفظ الرسومات
- في حال وجود أكثر من نتيجة، يمكن إنشاء DataFrame ملخص باسم results
-  حاول رسم النتائج اذا كان للرسمه معنا وكانت اكثر من قيمه النتيجه بالرسمه المناسبه مثال : السلاسل الزمنيه ترسم ب منحني خطي 
- إذا كان من المنطقي رسم النتائج، قم بكتابة كود Seaborn أو matplotlib المناسب 
-مهم تاكد قبل الرسم ان تختار اماكن متغيرات الرسمه بدقه 
"""

    messages = [
        {"role": "system", "content": "أنت مساعد يحول الأسئلة إلى كود Pandas و Seaborn أو matplotlib فقط حسب الحاجة مع مراعاة السياق."},
        {"role": "user", "content": prompt}
    ]
    client = Together(api_key=secret_value_0)
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1-0528",
        #model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages,
        temperature=.3
    )
    raw_output = response.choices[0].message.content.strip()
    return extract_code_from_response(raw_output)

def run_pandas_code(code: str, df):
    local_vars = {'df': df.copy(), 'plt': plt, 'sns': sns}
    try:
        exec(code, {}, local_vars)
        results = {k: v for k, v in local_vars.items() if k.startswith("result")}
        
        # Handle multiple figures
        plot_paths = []
        # Ensure 'plots' directory exists
        os.makedirs("plots", exist_ok=True) 
        for var_name, var_value in local_vars.items():
            if var_name.startswith("fig") and isinstance(var_value, plt.Figure):
                plot_filename = f"plots/{var_name}_{pd.Timestamp.now().strftime('%Y%m%d%H%M%S%f')}.png"
                var_value.savefig(plot_filename)
                plot_paths.append(plot_filename)
                plt.close(var_value) # Close the figure to free up memory
                
        return results, plot_paths
    
    except Exception as e:
        return f"⚠️ خطأ في تنفيذ الكود:\n{e}", []

def explain_results(question: str, code: str, results) -> str:
    prompt = f"""
السؤال: {question}

الكود المستخدم:
{code}

النتائج: {results}

اكتب شرحاً واضحاً وبسيطاً باللغة العربية يعكس النتائج الإحصائية فقط.
اجعل النص المخرجات مختصر 
- لا تُظهر خطوات التفكير أو التحليل الداخلي.
- استخدم لغة احترافية مباشرة موجهة للمستخدم غير المتخصص.
"""

    messages = [
        {
            "role": "system",
            "content": """   أنت خبير بيانات مهمتك شرح النتائج الإحصائية للمستخدمين باللغة العربية بشكل مختصر لكن احترافي اجعل االنتائج شكلها محترف" .
❌ لا تُظهر خطوات التفكير أو تستخدم عبارات مثل "لقد طلب المستخدم" أو "أفكر في...".
"""
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    client = Together(api_key=secret_value_0)
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1-0528",
        #model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages,
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

def process_question(question):
    try:
        best_file = choose_best_table(question)
        path = metadata_index[best_file]["path"]
        df = pd.read_excel(path, sheet_name="Relational DB", skiprows=2)
        df.columns = df.columns.str.strip()
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].str.strip()
        df = df.rename(columns={'Measure Name ': 'measure_name'})
        df.replace(["Not Applicable", "لا ينطبق", "Not App", "n.a.", "N/A"], pd.NA, inplace=True)

        exclude_cols = ['measure_name','قيمة المؤشر']
        cols_to_show = [col for col in df.columns if col not in exclude_cols]
        cols_info = ""
        for col in cols_to_show:
            unique_vals = df[col].dropna().unique()
            unique_vals_sample = unique_vals[:]
            unique_vals_str = ", ".join(map(str, unique_vals_sample))
            if len(unique_vals) > 20:
                unique_vals_str += ", ..."
            cols_info += f"- {col}: القيم الفريدة: [{unique_vals_str}]\n"

        code = generate_pandas_code(question, df_name="df", table_info=cols_info)
        results, plot_paths = run_pandas_code(code, df)

        explanation = "❌ لا يمكن شرح النتيجة بسبب وجود خطأ في الكود." if "خطأ" in str(results) else explain_results(question, code, results)
        
        # Gradio expects a list of paths for gr.Gallery
        return f"📝 الكود:\n{code}", f"📊 النتيجة:\n{results}", f"💡 التفسير:\n{explanation}", plot_paths
    except Exception as e:
        return "", f"❌ حدث خطأ أثناء المعالجة:\n{str(e)}", "", []

def launch_interface():
    iface = gr.Interface(
        fn=process_question,
        inputs=gr.Textbox(lines=2, label="📥 أدخل سؤالك بالعربية"),
        outputs=[
            gr.Textbox(label="كود الباندا و التصوير"),
            gr.Textbox(label="النتيجة أو الخطأ"),
            gr.Textbox(label="الشرح بالعربية"),
            gr.Gallery(label="📈 الرسومات البيانية", preview=True, height="auto") # Changed to gr.Gallery
        ],
        title="مساعد تحليلات Pandas 📊",
        description="ادخل سؤالك بالعربية لتحليل بيانات الجداول الإحصائية",
        theme="default"
    )
    iface.launch()

if __name__ == "__main__":
    launch_interface()

* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://fbf799201bed80b5d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


In [ ]:
#SAMPLE QUESTIONS
كم عدد المطلقات في كل محافظة لعامي 2020 و 2021 

قارن لي بين عدد المواليد في عمان و العقبه علي مر السنين 

كم عدد المواليد الذكور  والاناث في كل محافظة عام 2020

اعطني ملخص لجدول المواليد

اعطني ملخص لجدول عقود الزواج
